In [2]:
from utils.database import Postgres
import os
from utils.models import User
from dotenv import load_dotenv

load_dotenv()

# Create an instance of the Postgres class
postgress = Postgres(uri=os.getenv("POSTGRES_URI"), database=os.getenv("POSTGRES_DATABASE"))


In [14]:
user = User(email='dhr1@gmail.com', password='123456789')
postgress.cursor.execute(f"SELECT * FROM users WHERE email='{user.email}'")
user.name, user.email, user.password_hash, user.salt, user.gender, user.age, user.weight, user.height, user.goal, user.activity_level, user.joined = postgress.cursor.fetchone()

user.model_dump(exclude={'password', 'password_hash', 'salt', 'joined'})
# user.model_dump()
# type(user.joined.strftime("%Y-%m-%d"))


{'name': 'Dhr',
 'email': 'dhr1@gmail.com',
 'gender': 'Male',
 'age': 20,
 'weight': 60.0,
 'height': 200,
 'goal': 'Stay Fit',
 'activity_level': 'Pro'}

In [34]:
from pydantic import BaseModel, field_validator, ValidationError

class User(BaseModel):
    name: str

    @field_validator('name')
    def name_must_contain_space(cls, v):
        raise ValueError('$$ mai hu error $$')

    @classmethod
    def get_error_message(cls, exc: ValidationError) -> str:
        for error in exc.errors():
            if error['type'] == 'value_error':
                error_msg = error['msg']
                if error_msg.startswith('Value error, '):
                    return error_msg[len('Value error, '):]
                return error_msg
        return ''

try:
    user = User(name='John ')
except ValidationError as e:
    print(User.get_error_message(e))


$$ mai hu error $$


In [19]:
with open('utils/models.py', 'r') as f:
    print(f.readlines())


['from pydantic import BaseModel, field_validator, ValidationError\n', 'import re\n', 'import datetime\n', '\n', 'class User(BaseModel):\n', '    name: str = None\n', '    email: str\n', '    password: str\n', '    password_hash: bytes = None\n', '    salt: bytes = None\n', '    gender: str = None\n', '    age: int = None\n', '    weight: float = None\n', '    height: int = None\n', '    goal: str = None\n', '    activity_level: str = None\n', '    joining_date: str = None\n', '    \n', "    @field_validator('name')\n", '    def validate_name(cls, name: str):\n', '        if len(name) < 3:\n', '            raise ValueError("Name must be at least 3 characters")\n', '        return name.title()\n', '    \n', "    @field_validator('email')\n", '    def validate_email(cls, email: str):\n', '        if not re.match(r"[^@]+@[^@]+\\.[^@]+", email):\n', '            raise ValueError("Invalid email address")\n', '        return email\n', '    \n', "    @field_validator('password')\n", '    def 